In [1]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nglview as nv
import MDAnalysis as mda
from MDAnalysis.analysis import rms, diffusionmap, align

Folder with MD Simulations

In [2]:
HERE = Path(_dh[-1])
DATA = HERE.parent / "data"
MDSIMS = DATA / "pyrod_acceptor" / "sim"

Variable setting

In [8]:
d = {'Molecule': [], 'Max RMSF': [], 'Max RMSD' : []}
df = pd.DataFrame(data=d)
universes = []

In [4]:
for folder in os.scandir(MDSIMS):
    u = mda.Universe(str(MDSIMS / folder / "topology_wrapped.pdb"), str(MDSIMS / folder / "trajectory_wrapped.xtc"))
    average = align.AverageStructure(u, u, select='protein and name CA',
                                 ref_frame=0).run()
    ref = average.universe
    aligned_traj = str(MDSIMS / folder / "trajectory_aligned.xtc")
    aligner = align.AlignTraj(u, ref,
                              select='protein and name CA',
                              filename=aligned_traj,
                              in_memory=False).run()
    print(f'Aligned MD sim for Molecule {str(folder)}')

Aligned MD sim for Molecule <DirEntry 'MolPort-002-575-131_1_2_1_1.sdf'>
Aligned MD sim for Molecule <DirEntry 'MolPort-045-903-060_1_1_1_1.sdf'>
Aligned MD sim for Molecule <DirEntry 'MolPort-020-106-807_1_1_1_1.sdf'>
Aligned MD sim for Molecule <DirEntry 'MolPort-044-384-097_1_4_1_1.sdf'>
Aligned MD sim for Molecule <DirEntry 'MolPort-000-883-333_1_1_1_1.sdf'>
Aligned MD sim for Molecule <DirEntry 'MolPort-008-339-740_1_2_1_1.sdf'>
Aligned MD sim for Molecule <DirEntry 'MolPort-004-332-156_1_1_1_1.sdf'>
Aligned MD sim for Molecule <DirEntry 'MolPort-004-949-002_1_3_1_1.sdf'>
Aligned MD sim for Molecule <DirEntry 'MolPort-005-066-645_1_1_1_1.sdf'>
Aligned MD sim for Molecule <DirEntry 'MolPort-002-465-212_1_2_1_1.sdf'>
Aligned MD sim for Molecule <DirEntry 'MolPort-008-342-579_1_1_1_1.sdf'>
Aligned MD sim for Molecule <DirEntry 'MolPort-008-337-141_1_3_1_1.sdf'>
Aligned MD sim for Molecule <DirEntry 'MolPort-002-605-763_1_2_1_1.sdf'>
Aligned MD sim for Molecule <DirEntry 'MolPort-023-

In [4]:
for folder in os.scandir(MDSIMS):
    u = mda.Universe(str(MDSIMS / folder / "topology_wrapped.pdb"), str(MDSIMS / folder / "trajectory_aligned.xtc"))
    R = rms.RMSF(u.select_atoms("resname Mol")).run()
    df.loc[len(df.index)] = [folder,np.max(R.rmsf)]

In [5]:
def rmsd_for_atomgroups(universe):
    """Calulate the RMSD for selected atom groups.

    Parameters
    ----------
    universe: MDAnalysis.core.universe.Universe
        MDAnalysis universe.
    selection1: str
        Selection string for main atom group, also used during alignment.
    selection2: list of str, optional
        Selection strings for additional atom groups.

    Returns
    -------
    rmsd_df: pandas.core.frame.DataFrame
        DataFrame containing RMSD of the selected atom groups over time.
    """

    universe.trajectory[0]
    ref = universe
    rmsd_analysis = rms.RMSD(universe, ref, select="resname Mol")
    rmsd_analysis.run()
    columns = ["Mol"]
    rmsd_df = pd.DataFrame(np.round(rmsd_analysis.rmsd[:, 2:], 2), columns=columns)
    rmsd_df.index.name = "frame"
    return rmsd_df

In [9]:
for folder in os.scandir(MDSIMS):
    u = mda.Universe(str(MDSIMS / folder / "topology_wrapped.pdb"), str(MDSIMS / folder / "trajectory_aligned.xtc"))
    rmsd = rmsd_for_atomgroups(u)
    R = rms.RMSF(u.select_atoms("resname Mol")).run()
    #print(f'{str(folder)}: {np.max(rmsd)}')
    df.loc[len(df.index)] = [folder,np.round(np.max(R.rmsf),2),np.max(np.max(rmsd))]

In [10]:
df

,Molecule,Max RMSF,Max RMSD
0,<DirEntry 'MolPort-002-575-131_1_2_1_1.sdf'>,3.495900,2.42
1,<DirEntry 'MolPort-045-903-060_1_1_1_1.sdf'>,2.649636,1.71
2,<DirEntry 'MolPort-020-106-807_1_1_1_1.sdf'>,2.523851,3.00
3,<DirEntry 'MolPort-044-384-097_1_4_1_1.sdf'>,1.602531,1.29
4,<DirEntry 'MolPort-000-883-333_1_1_1_1.sdf'>,2.056778,1.38
5,<DirEntry 'MolPort-008-339-740_1_2_1_1.sdf'>,2.253986,1.12
6,<DirEntry 'MolPort-004-332-156_1_1_1_1.sdf'>,3.444668,2.25
7,<DirEntry 'MolPort-004-949-002_1_3_1_1.sdf'>,3.094939,2.01
8,<DirEntry 'MolPort-005-066-645_1_1_1_1.sdf'>,1.696962,1.13
9,<DirEntry 'MolPort-002-465-212_1_2_1_1.sdf'>,2.456250,1.71
